In [73]:
import pandas as pd
import numpy as np
import os
import csv
import os.path

In [234]:
working_dir = os.path.abspath(os.path.dirname('data_4_ScienceHack2021'))

In [235]:
working_dir

'/Users/nikanizharadze/Desktop/data_4_ScienceHack2021'

In [216]:
folder_name = 'trip_'
gps_filename = 'gps_data.csv'
passanger_count_filename = 'passenger_count_nums'
all_trips = []
# for i in range(1,13):
for i in range(1,13):
    df_gps = pd.read_csv(working_dir+folder_name+str(i) +'/' + gps_filename)
    df_passanger_count = pd.read_csv(working_dir+folder_name+str(i) +'/' + passanger_count_filename)
    # Remove Nan, inf values in passanger_count file
    df_passanger_count = df_passanger_count[~df_passanger_count.isin([np.nan, np.inf, -np.inf]).any(1)]
    # # Convert epoch_ts to int so GPS dataframe and Passanger_Count dataframe are mergable on epoch_ts
    df_passanger_count['Epoch_Time'] = df_passanger_count['Epoch_Time'].astype(float).round().astype(int)
    df_passanger_count = df_passanger_count.rename(columns={'Epoch_Time': 'epoch_ts'})
    merged = df_gps.merge(df_passanger_count, on="epoch_ts", how='left')
    merged = merged.fillna(0)
    door_closed_indeces = merged[merged.Number_of_Passengers==-10].index.values
    door_open_indeces = merged[merged.Number_of_Passengers==-99].index.values
    for j in range(len(door_closed_indeces)):
        value_to_insert = sum(merged.iloc[door_closed_indeces[j]+1:door_open_indeces[j]-1]['Number_of_Passengers'])
        merged.loc[door_closed_indeces[j]+1:door_open_indeces[j]-1]['Number_of_Passengers'] =  value_to_insert   
    all_trips.append(merged)

/Users/nikanizharadze/anaconda3/envs/PricePrediction/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [221]:
len(all_trips)

12

In [236]:
# all_trips contain 12 dataframes each of which merges relevant gps data to number of passengers data based on epoch_ts

In [237]:
all_trips[0].head(50)

,date_time,is_valid,latitude,lat_dir,longitude,lon_dir,speed_over_ground_knots,heading_true_course,epoch_ts,Number_of_Passengers
0,2021-04-12 09:22:22,False,0.000000,0,0.000000,0,0.000,0.00,1.618219e+09,0.0
1,2021-04-12 09:22:23,True,48.151783,N,11.525906,E,19.488,260.93,1.618219e+09,0.0
2,2021-04-12 09:22:24,True,48.151754,N,11.525763,E,19.709,259.68,1.618219e+09,0.0
3,2021-04-12 09:22:25,True,48.151737,N,11.525618,E,19.987,260.13,1.618219e+09,0.0
4,2021-04-12 09:22:26,True,48.151743,N,11.525471,E,20.009,263.38,1.618219e+09,0.0
5,2021-04-12 09:22:27,True,48.151725,N,11.525324,E,20.273,263.36,1.618219e+09,0.0
6,2021-04-12 09:22:28,True,48.151676,N,11.525196,E,19.985,260.49,1.618219e+09,-10.0
7,2021-04-12 09:22:29,True,48.151645,N,11.525047,E,20.596,259.42,1.618219e+09,12.0
8,2021-04-12 09:22:30,True,48.151616,N,11.524912,E,20.281,258.78,1.618219e+09,12.0
9,2021-04-12 09:22:31,True,48.151592,N,11.524777,E,19.905,259.07,1.618219e+09,12.0
